##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 환경

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/2_environments_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/2_environments_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/2_environments_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/2_environments_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

## 소개

RL(Reinforcement Learning)의 목표는 환경과 상호 작용하여 학습하는 에이전트를 설계하는 것입니다. 표준 RL 설정에서 에이전트는 타임스텝마다 관측 값을 수신하고 행동을 선택합니다. 행동이 환경에 적용되고 환경이 보상과 새로운 관찰 값을 반환합니다. 에이전트는 이익이라고도 하는 보상의 합계를 최대화하기 위한 행동을 선택하도록 정책을 훈련합니다.

TF-Agents에서 환경은 Python 또는 TensorFlow로 구현될 수 있습니다. Python 환경은 일반적으로 구현, 이해 및 디버깅하기 더 쉽지만, TensorFlow 환경은 더 효율적이며 자연적인 병렬화가 가능합니다. 가장 일반적인 워크플로는 Python에서 환경을 구현하고 래퍼 중 하나를 사용하여 환경을 TensorFlow로 자동 변환합니다.

Python 환경을 먼저 살펴보겠습니다. TensorFlow 환경은 매우 유사한 API를 따릅니다.

## 설정


tf-agents 또는 gym을 아직 설치하지 않은 경우, 다음을 실행합니다.

In [ ]:
!pip install "gym>=0.21.0"
!pip install tf-agents


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

## Python 환경

Python 환경에는 행동을 환경에 적용하는 `step(action) -> next_time_step` 메서드가 있으며 다음 단계에 대한 다음 정보를 반환합니다.

1. `observation`: 에이전트가 다음 단계에서 행동을 선택하기 위해 관찰할 수 있는 환경 상태의 일부입니다.
2. `reward`: 에이전트는 여러 단계에 걸쳐 이러한 보상의 합계를 최대화하는 방법을 학습하고 있습니다.
3. `step_type`: 환경과의 상호 작용은 일반적으로 시퀀스/에피소드의 일부입니다. 체스 게임에서 여러 번의 이동을 예로 들 수 있습니다. step_type은 `FIRST`, `MID` 또는 `LAST`일 수 있으며, 해당 타임스텝이 시퀀스의 첫 번째 단계인지 중간 단계인지 마지막 단계인지를 나타냅니다.
4. `discount`: 현재 타임스텝의 보상에 대한 다음 타임스텝의 보상 가중치를 나타내는 부동 소수점입니다.

이들은 명명된 튜플 `TimeStep(step_type, reward, discount, observation)`으로 그룹화됩니다.

The interface that all Python environments must implement is in `environments/py_environment.PyEnvironment`. The main methods are:

In [ ]:
class PyEnvironment(object):

  def reset(self):
    """Return initial_time_step."""
    self._current_time_step = self._reset()
    return self._current_time_step

  def step(self, action):
    """Apply action and return new time_step."""
    if self._current_time_step is None:
        return self.reset()
    self._current_time_step = self._step(action)
    return self._current_time_step

  def current_time_step(self):
    return self._current_time_step

  def time_step_spec(self):
    """Return time_step_spec."""

  @abc.abstractmethod
  def observation_spec(self):
    """Return observation_spec."""

  @abc.abstractmethod
  def action_spec(self):
    """Return action_spec."""

  @abc.abstractmethod
  def _reset(self):
    """Return initial_time_step."""

  @abc.abstractmethod
  def _step(self, action):
    """Apply action and return new time_step."""

`step()` 메서드 외에도, 환경은 새 시퀀스를 시작하고 초기 `TimeStep`을 제공하는 `reset()` 메서드도 제공합니다. `reset` 메서드를 명시적으로 호출할 필요는 없습니다. 에피소드가 끝나거나 step()이 처음 호출될 때 환경이 자동으로 재설정된다고 가정합니다.

서브 클래스는 `step()` 또는 `reset()`을 직접 구현하지 않습니다. 대신 `_step()` 및 `_reset()` 메서드를 재정의합니다. 이들 메서드에서 반환된 타임스텝은 `current_time_step()`를 통해 캐시되고 노출됩니다.

`observation_spec` 및 `action_spec` 메서드는 관찰 값 및 행동의 이름, 형상, 데이터 유형 및 범위를 각각 설명하는 `(Bounded)ArraySpecs`의 중첩을 반환합니다.

TF-Agents에서 목록, 튜플, 명명된 튜플 또는 사전으로 구성된 구조와 같은 트리로 정의된 중첩을 반복해서 참조합니다. 중첩은 관찰 값과 행동의 구조를 유지하기 위해 임의로 구성될 수 있습니다. 관찰 값과 행동이 많은 복잡한 환경에 매우 유용한 것으로 나타났습니다.

### 표준 환경 사용하기

TF Agents에는 OpenAI Gym, DeepMind-control 및 Atari와 같은 많은 표준 환경을 위한 래퍼가 내장되어 있으며 `py_environment.PyEnvironment` 인터페이스를 따릅니다. 이들 래핑된 환경은 환경 도구 모음를 사용하여 쉽게 로드할 수 있습니다. OpenAI gym에서 CartPole 환경을 로드하고 행동과 time_step_spec을 살펴보겠습니다.

In [ ]:
environment = suite_gym.load('CartPole-v0')
print('action_spec:', environment.action_spec())
print('time_step_spec.observation:', environment.time_step_spec().observation)
print('time_step_spec.step_type:', environment.time_step_spec().step_type)
print('time_step_spec.discount:', environment.time_step_spec().discount)
print('time_step_spec.reward:', environment.time_step_spec().reward)


환경이 [0, 1]에서 유형 `int64`의 행동을 기대하고 `TimeSteps`를 반환합니다. 관찰 값은 길이 4의 `float32` 벡터이며, 할인 인자는 [0.0, 1.0]에서 `float32`입니다. 이제 전체 에피소드에 대해 고정된 행동 `(1,)`을 취해 보겠습니다.

In [ ]:
action = np.array(1, dtype=np.int32)
time_step = environment.reset()
print(time_step)
while not time_step.is_last():
  time_step = environment.step(action)
  print(time_step)

### 자신만의 Python 환경 만들기

많은 고객에게 일반적인 사용 사례는 TF-Agents의 표준 에이전트(agents/ 참조) 중 하나를 문제에 적용하는 것입니다. 이를 위해서는 문제를 환경으로 만들어야 합니다. Python으로 환경을 구현하는 방법을 살펴보겠습니다.

에이전트가 다음과 같은 카드 게임((블랙 잭에서 영감받음)을 플레이하도록 훈련하고 싶다고 가정해 보겠습니다.

1. 이 게임은 1에서 10까지의 무한한 카드 한 벌을 사용하여 진행됩니다.
2. 매번 에이전트는 두 가지 일을 할 수 있습니다. 새로운 임의의 카드를 얻거나 현재 라운드를 중단합니다.
3. 목표는 라운드가 끝날 때 카드의 합계를 가능한 한 21에 가깝게 유지하는 것입니다.

게임을 나타내는 환경은 다음과 같습니다.

1. Actions: 2 가지 행동이 있습니다. Action 0: 새 카드를 받고, Action 1: 현재 라운드를 종료합니다.
2. Observations: 현재 라운드에서의 카드의 합계
3. Reward: 목표는 21을 넘어가지 않고 가능한 한 21에 가까워지는 것이므로 라운드가 끝날 때 다음 보상을 사용하여 이를 달성할 수 있습니다. sum_of_cards - 21 if sum_of_cards &lt;= 21, else -21


In [ ]:
class CardGameEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, name='observation')
    self._state = 0
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = 0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Make sure episodes don't go on forever.
    if action == 1:
      self._episode_ended = True
    elif action == 0:
      new_card = np.random.randint(1, 11)
      self._state += new_card
    else:
      raise ValueError('`action` should be 0 or 1.')

    if self._episode_ended or self._state >= 21:
      reward = self._state - 21 if self._state <= 21 else -21
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

위의 환경을 올바르게 정의하여 모든 작업을 수행했는지 확인합니다. 자신의 환경을 만들 때, 생성된 관찰 값 및 time_steps가 사양에 정의된 올바른 형상 및 유형을 따르는지 확인해야 합니다. 이들은 TensorFlow 그래프를 생성하는 데 사용되므로 잘못 정의하면 디버깅하기 어려운 문제가 발생할 수 있습니다.

환경을 검증하기 위해 임의의 정책을 사용하여 행동을 생성하고 5가지 이상의 에피소드를 반복하여 환경이 의도한 대로 동작하는지 확인합니다. 환경 사양을 따르지 않는 time_step을 수신하면 오류가 발생합니다.

In [ ]:
environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=5)

이제 환경이 의도한 대로 동작하고 있음을 알았으므로 고정된 정책을 사용하여 이 환경을 실행하겠습니다. 3장의 카드를 요청한 후 라운드를 종료합니다.

In [ ]:
get_new_card_action = np.array(0, dtype=np.int32)
end_round_action = np.array(1, dtype=np.int32)

environment = CardGameEnv()
time_step = environment.reset()
print(time_step)
cumulative_reward = time_step.reward

for _ in range(3):
  time_step = environment.step(get_new_card_action)
  print(time_step)
  cumulative_reward += time_step.reward

time_step = environment.step(end_round_action)
print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

### 환경 래퍼

An environment wrapper takes a Python environment and returns a modified version of the environment. Both the original environment and the modified environment are instances of `py_environment.PyEnvironment`, and multiple wrappers can be chained together.

일부 일반적인 래퍼는 `environments/wrappers.py`에서 찾을 수 있습니다. 예를 들면, 다음과 같습니다.

1. `ActionDiscretizeWrapper`: 연속 행동 공간을 불연속 행동 공간으로 변환합니다.
2. `RunStats`: 수행한 단계 수, 완료된 에피소드 수 등과 같은 환경의 실행 통계를 캡처합니다.
3. `TimeLimit`: 정해진 수의 단계 후에 에피소드를 종료합니다.


#### 예 1: 행동 불연속화 래퍼

InvertedPendulum은 범위 `[-2, 2]`에서 연속 행동을 허용하는 PyBullet 환경입니다. 이 환경에서 DQN과 같은 불연속 행동 에이전트를 훈련하려면 행동 공간을 불연속화(양자화)해야 합니다. 이것이 바로 `ActionDiscretizeWrapper`가 하는 일입니다. 래핑 전후의 `action_spec`을 비교합니다.

In [ ]:
env = suite_gym.load('Pendulum-v1')
print('Action Spec:', env.action_spec())

discrete_action_env = wrappers.ActionDiscretizeWrapper(env, num_actions=5)
print('Discretized Action Spec:', discrete_action_env.action_spec())

The wrapped `discrete_action_env` is an instance of `py_environment.PyEnvironment` and can be treated like a regular Python environment.


## TensorFlow Environments

The interface for TF environments is defined in `environments/tf_environment.TFEnvironment` and looks very similar to the Python environments. TF Environments differ from Python envs in a couple of ways:

- 배열 대신 텐서 객체를 생성합니다.
- TF Environments는 사양과 비교하여 생성된 텐서에 배치 차원을 추가합니다.

Converting the Python environments into TFEnvs allows tensorflow to parallelize operations. For example, one could define a `collect_experience_op` that collects data from the environment and adds to a `replay_buffer`, and a `train_op` that reads from the `replay_buffer` and trains the agent, and run them in parallel naturally in TensorFlow.

In [ ]:
class TFEnvironment(object):

  def time_step_spec(self):
    """Describes the `TimeStep` tensors returned by `step()`."""

  def observation_spec(self):
    """Defines the `TensorSpec` of observations provided by the environment."""

  def action_spec(self):
    """Describes the TensorSpecs of the action expected by `step(action)`."""

  def reset(self):
    """Returns the current `TimeStep` after resetting the Environment."""
    return self._reset()

  def current_time_step(self):
    """Returns the current `TimeStep`."""
    return self._current_time_step()

  def step(self, action):
    """Applies the action and returns the new `TimeStep`."""
    return self._step(action)

  @abc.abstractmethod
  def _reset(self):
    """Returns the current `TimeStep` after resetting the Environment."""

  @abc.abstractmethod
  def _current_time_step(self):
    """Returns the current `TimeStep`."""

  @abc.abstractmethod
  def _step(self, action):
    """Applies the action and returns the new `TimeStep`."""

`current_time_step()` 메서드는 현재 time_step을 반환하고 필요한 경우 환경을 초기화합니다.

`reset()` 메서드는 환경에서 강제로 재설정하고 current_step을 반환합니다.

`action`이 이전 `time_step`에 의존하지 않으면 `tf.control_dependency`가 `Graph` 모드에 필요합니다.

지금은 `TFEnvironments`를 만드는 방법을 살펴보겠습니다.

### 자신만의 TensorFlow Environment 만들기

이것은 Pytohn에서 환경을 만드는 것보다 더 복잡하므로 이 colab에서는 다루지 않습니다. [여기](https://github.com/tensorflow/agents/blob/master/tf_agents/environments/tf_environment_test.py)에서 예제를 볼 수 있습니다. 더 일반적인 사용 사례는 Python으로 환경을 구현하고 `TFPyEnvironment` 래퍼를 사용하여 환경을 TensorFlow로 래핑하는 것입니다(아래 참조).

### TensorFlow에서 Python Environment 래핑하기

`TFPyEnvironment` 래퍼를 사용하여 모든 Python 환경을 TensorFlow 환경으로 쉽게 래핑할 수 있습니다.

In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

print(isinstance(tf_env, tf_environment.TFEnvironment))
print("TimeStep Specs:", tf_env.time_step_spec())
print("Action Specs:", tf_env.action_spec())

이제 사양의 유형은 `(Bounded)TensorSpec`입니다.

### 사용 예

#### 간단한 예제

In [ ]:
env = suite_gym.load('CartPole-v0')

tf_env = tf_py_environment.TFPyEnvironment(env)
# reset() creates the initial time_step after resetting the environment.
time_step = tf_env.reset()
num_steps = 3
transitions = []
reward = 0
for i in range(num_steps):
  action = tf.constant([i % 2])
  # applies the action and returns the new TimeStep.
  next_time_step = tf_env.step(action)
  transitions.append([time_step, action, next_time_step])
  reward += next_time_step.reward
  time_step = next_time_step

np_transitions = tf.nest.map_structure(lambda x: x.numpy(), transitions)
print('\n'.join(map(str, np_transitions)))
print('Total reward:', reward.numpy())

#### 전체 에피소드

In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)